[View in Colaboratory](https://colab.research.google.com/github/AvivSham/YOLO_V3_from_scratch_colab/blob/master/Detection.ipynb)

###Here you need to load darknet & util modules  in addition to the pallete file

In [0]:
#@title Installing PyTorch 
!pip install torch==0.4


In [0]:
#@title Import dependencies
from __future__ import division
import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
from util import *
import argparse
import os 
import os.path as osp
from darknet import *
import pickle as pkl
import pandas as pd
import random
import csv

In [0]:
#@title Get weights and graph file for the model

#Get weights file for the model
!wget https://pjreddie.com/media/files/yolov3.weights
# Get YOLO graph file
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg


In [0]:
#@title Making directories for test photos
!mkdir Images
!mkdir Images_Detection

In [0]:
#@title Changing working directory
os.chdir(os.getcwd()+'/'+'Images')

In [6]:
#@title Test
os.getcwd()

'/content/Images'

In [0]:
#@title Get test images
!wget https://github.com/ayooshkathuria/pytorch-yolo-v3/raw/master/dog-cycle-car.png
!wget https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/imgs/giraffe.jpg
!wget https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/imgs/messi.jpg
!wget https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/imgs/img1.jpg
!wget https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/imgs/img2.jpg
!wget https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/imgs/img3.jpg
!wget https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/imgs/img4.jpg


In [0]:
%cd ..


In [0]:
#@title Setting parameters


# images --> Input image / directory of images.
# reso   --> Input image resolution
# detect_image --> flag to decide to detect video or single photo

images = os.getcwd() + '/Images'
images_det = os.getcwd() + '/Images_Detection'
print(images, images_det)
batch_size = 1
confidence = 0.5
nms_thesh = 0.4
reso = 416
weights_file = "yolov3.weights"
config_file = "yolov3.cfg"
start = 0
detect_image = True
CUDA = torch.cuda.is_available()

In [0]:
#@title Making directory for COCO names file
!mkdir data
os.chdir('./data')



In [0]:
#@title COCO names file
!wget https://raw.githubusercontent.com/ayooshkathuria/YOLO_v3_tutorial_from_scratch/master/data/coco.names

In [0]:
def load_classes(namefile):
  fp = open(namefile, 'r')
  names = fp.read().split("\n")[:-1] #discard the last
  return names

In [0]:
# COCO dataset has 80 different classes
num_classes = 80
classes = load_classes("coco.names")

In [0]:
%cd ..

In [0]:
# Set up the network
print("Loading network...")
model = Darknet(config_file)
model.load_weights(weights_file)
print("Network successfully loaded")


model.net_info["height"] = reso
inp_dim = int(model.net_info["height"])
assert inp_dim % 32 == 0
assert inp_dim > 32


def write_box(x, results):
    c1 = tuple(x[1:3].int())
    c2 = tuple(x[3:5].int())
    img = results[int(x[0])]
    cls = int(x[-1])
    color = random.choice(colors)
    label = "{0}".format(classes[cls])
    cv2.rectangle(img, c1, c2,color, 2)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
    return img



# if GPU is available allocate the model
if CUDA:
  model.cuda()
  
# Set the model in evaluation mode
model.eval()

if detect_image:
  read_dir = time.time()
  #Detection phase
  try:
      imlist = [osp.join(osp.realpath('.'), images, img) for img in os.listdir(images)]
  except NotADirectoryError:
      imlist = []
      imlist.append(osp.join(osp.realpath('.'), images))
  except FileNotFoundError:
      print ("No file or directory with the name {}".format(images))
      exit()

  if not os.path.exists(images_det):
      os.makedirs(images_det)

  load_batch = time.time()
  loaded_ims = [cv2.imread(x) for x in imlist]

  im_batches = list(map(prep_image, loaded_ims, [inp_dim for x in range(len(imlist))]))
  im_dim_list = [(x.shape[1], x.shape[0]) for x in loaded_ims]
  im_dim_list = torch.FloatTensor(im_dim_list).repeat(1,2)


  leftover = 0
  if (len(im_dim_list) % batch_size):
      leftover = 1

  if batch_size != 1:
      num_batches = len(imlist) // batch_size + leftover            
      im_batches = [torch.cat((im_batches[i*batch_size : min((i +  1)*batch_size,
                          len(im_batches))]))  for i in range(num_batches)]  

  write = 0


  if CUDA:
      im_dim_list = im_dim_list.cuda()

  start_det_loop = time.time()
  for i, batch in enumerate(im_batches):
  #load the image 
      start = time.time()
      if CUDA:
          batch = batch.cuda()
      with torch.no_grad():
          prediction = model(Variable(batch), CUDA)

      prediction = write_results(prediction, confidence, num_classes, nms_conf = nms_thesh)

      end = time.time()

      if type(prediction) == int:

          for im_num, image in enumerate(imlist[i*batch_size: min((i +  1)*batch_size, len(imlist))]):
              im_id = i*batch_size + im_num
              print("{0:20s} predicted in {1:6.3f} seconds".format(image.split("/")[-1], (end - start)/batch_size))
              print("{0:20s} {1:s}".format("Objects Detected:", ""))
              print("----------------------------------------------------------")
          continue

      prediction[:,0] += i*batch_size    #transform the atribute from index in batch to index in imlist 

      if not write:                      #If we have't initialised output
          output = prediction  
          write = 1
      else:
          output = torch.cat((output,prediction))

      for im_num, image in enumerate(imlist[i*batch_size: min((i +  1)*batch_size, len(imlist))]):
          im_id = i*batch_size + im_num
          objs = [classes[int(x[-1])] for x in output if int(x[0]) == im_id]
          print("{0:20s} predicted in {1:6.3f} seconds".format(image.split("/")[-1], (end - start)/batch_size))
          print("{0:20s} {1:s}".format("Objects Detected:", " ".join(objs)))
          print("----------------------------------------------------------")

      if CUDA:
          torch.cuda.synchronize()       
  try:
      output
  except NameError:
      print ("No detections were made")
      exit()

  im_dim_list = torch.index_select(im_dim_list, 0, output[:,0].long())

  scaling_factor = torch.min(416/im_dim_list,1)[0].view(-1,1)


  output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim_list[:,0].view(-1,1))/2
  output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim_list[:,1].view(-1,1))/2



  output[:,1:5] /= scaling_factor

  for i in range(output.shape[0]):
      output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim_list[i,0])
      output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim_list[i,1])


  output_recast = time.time()
  class_load = time.time()
  colors = pkl.load(open("pallete", "rb"))

  draw = time.time()
  
  list(map(lambda x: write_box(x, loaded_ims), output))

  det_names = pd.Series(imlist).apply(lambda x: "{}/det_{}".format(images_det,x.split("/")[-1]))

  list(map(cv2.imwrite, det_names, loaded_ims))


  end = time.time()

  print("SUMMARY")
  print("----------------------------------------------------------")
  print("{:25s}: {}".format("Task", "Time Taken (in seconds)"))
  print()
  print("{:25s}: {:2.3f}".format("Reading addresses", load_batch - read_dir))
  print("{:25s}: {:2.3f}".format("Loading batch", start_det_loop - load_batch))
  print("{:25s}: {:2.3f}".format("Detection (" + str(len(imlist)) +  " images)", output_recast - start_det_loop))
  print("{:25s}: {:2.3f}".format("Output Processing", class_load - output_recast))
  print("{:25s}: {:2.3f}".format("Drawing Boxes", end - draw))
  print("{:25s}: {:2.3f}".format("Average time_per_img", (end - load_batch)/len(imlist)))
  print("----------------------------------------------------------")


  torch.cuda.empty_cache()

else:
  #Detection phase
  cap = cv2.VideoCapture('Nature.mp4')

  #cap = cv2.VideoCapture(0)  for webcam

  assert cap.isOpened(), 'Cannot capture source'

  frames = 0 
  video_detect = []
  start = time.time()

  while cap.isOpened():
      ret, frame = cap.read()

      if ret:   
          img = prep_image(frame, inp_dim)
  #        cv2.imshow("a", frame)
          im_dim = frame.shape[1], frame.shape[0]
          im_dim = torch.FloatTensor(im_dim).repeat(1,2)   

          if CUDA:
              im_dim = im_dim.cuda()
              img = img.cuda()

          with torch.no_grad():
              output = model(Variable(img, volatile = True), CUDA)
          output = write_results(output, confidence, num_classes, nms_conf = nms_thesh)


          if type(output) == int:
              frames += 1
              #cv2.imshow("frame", frame)
              #video_detect.append(frame)
              key = cv2.waitKey(1)
              if key & 0xFF == ord('q'):
                  break
              continue

          im_dim = im_dim.repeat(output.size(0), 1)
          scaling_factor = torch.min(416/im_dim,1)[0].view(-1,1)

          output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim[:,0].view(-1,1))/2
          output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim[:,1].view(-1,1))/2

          output[:,1:5] /= scaling_factor

          for i in range(output.shape[0]):
              output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim[i,0])
              output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim[i,1])


          classes = load_classes('data/coco.names')
          colors = pkl.load(open("pallete", "rb"))

          list(map(lambda x: write_box(x, frame), output))

          video_detect.append(frame)
          key = cv2.waitKey(1)
          if key & 0xFF == ord('q'):
              FPS = frames // (time.time() - start)
              break
          frames += 1
          
      else:
          FPS = frames // (time.time() - start)
          break
  

In [0]:
  def write(x, results):
      c1 = tuple(x[1:3].int())
      c2 = tuple(x[3:5].int())
      img = results[int(x[0])]
      cls = int(x[-1])
      color = random.choice(colors)
      label = "{0}".format(classes[cls])
      cv2.rectangle(img, c1, c2,color, 1)
      t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
      c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
      cv2.rectangle(img, c1, c2,color, -1)
      cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
      return img


In [0]:
#@title Further work
"""
video_detection_save = cv2.VideoWriter('video.avi',-1,20,(416,416))
for j in range(frames):
  video_detection_save.write(np.asarray(video_detect[j],dtype = np.uint8))
  
cv2.destroyAllWindows()
video_detection_save.release()
"""